# Requête HTTP 

Un requête HTTP est une requête basée sur le protocole TCP, elle fait partie de la couche application de la couche OSI. Elle permet d'accéder aux données mise à disposition sur une adresse IP (ou url résolue par un DNS) et un port. 

Les deux ports les plus utilisés dans le web sont le 80 pour les sites en HTTP et le 443 pour les sites en HTTPS. HTTPS est une variable du protocole HTTP basé sur le protocole TLS.

Il existe de nombreux types de requêtes selon la convention `REST`: 
- GET
- POST
- PUT 
- DELETE
- UPDATE.

Dans notre cas, nous allons utiliser la plupart du temps des GET et potentiellement des POST. 
- Le GET permet comme son nom l'indique de récupérer des informations en fonction de certains paramètres. 
- Le POST nécessite un envoi de données pour récupérer des données. Le body du post est, la plupart du temps, envoyé sous la forme d'un objet JSON.

Ces requêtes encapsulent un certain nombre de paramètres qui permettent soient d'identifier une provenance et un utilisateur ou de réaliser différentes actions.

In [2]:
import requests

In [3]:
url = "https://www.esiee.fr/"
response = requests.get(url)
response.status_code

200

Il existe deux méthodes pour récupérer le contenu de la page :

- `response.text` qui permet de retourner le texte sous la forme d'une chaine de charactères.
- `response.content` qui permet de récupérer le contenu de la page sous la forme de bytes

In [4]:
type(response.content)

bytes

In [5]:
type(response.text)

str

Pour récupérer les 1000 premiers charactères de la page :

In [6]:
response.text[0:1000]

'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2025 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;école de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande école d&#039;ingénieur dans les domaines des transitions numérique, énergétique et environnementale. Classée dans le groupe A, parmi les meilleures écoles d&#039;ingénieur selon le classement de l&#039;Etudiant. Habilitée par la Commission des Titres d&#039;Ingénieur (CTI). Membre de la Conférence des Grandes Ecoles (CGE). " />\n<meta name="viewport" conte

Pour récupérer les headers HTTP de la réponse :

In [7]:
response.headers

{'Date': 'Tue, 18 Nov 2025 13:27:20 GMT', 'Server': 'Apache', 'Content-Language': 'fr', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'X-UA-Compatible': 'IE=edge', 'X-Content-Type-Options': 'nosniff', 'Content-Length': '16642', 'Content-Type': 'text/html; charset=utf-8', 'X-Varnish': '533630589 535791364', 'Age': '73', 'Via': '1.1 varnish (Varnish/7.1)', 'Accept-Ranges': 'bytes', 'Connection': 'keep-alive'}

On peut modifier les paramètres de la requête et/ou ses headers. On peut par exemple ajouter un UserAgent (identifiant de l'initiateur de la requête) et un timeout de 10 secondes :

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(url, headers=headers, timeout = 10)
response.content[0:1000]

b'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2025 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;\xc3\xa9cole de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande \xc3\xa9cole d&#039;ing\xc3\xa9nieur dans les domaines des transitions num\xc3\xa9rique, \xc3\xa9nerg\xc3\xa9tique et environnementale. Class\xc3\xa9e dans le groupe A, parmi les meilleures \xc3\xa9coles d&#039;ing\xc3\xa9nieur selon le classement de l&#039;Etudiant. Habilit\xc3\xa9e par la Commission des Titres d&#039;Ing\xc3\xa9nieur (CTI). Membr

## Exercice

## Exercice 1

- Créer une classe Python permettant de faire des requêtes HTTP.
- Cette classe doit utiliser toujours le même UserAgent.
- Le TimeOut sera spécifié à chaque appelle avec une valeur par défaut.
- Un mécanisme de retry sera mis en place de façon recursive.


In [ ]:
class HttpClient:
    def __init__(self, user_agent: str = "MonSuperScraper/1.0"):
        """
        Constructeur de la classe, on définit ici le user agent qui sera utilisé pour toutes les requêtes
        """
        self.user_agent = user_agent
        self.headers = {"user-agent": self.user_agent}

    def get(self, url: str, timeout: float = 5.0, retries: int = 3):
        """
        Effectue une requête HTTP GET avec :
        - un timeout spécifiable
        - un mécanisme de retry récursif

        url: URL à appeler
        timeout: durée max d'attente en secondes
        retries: nombre de tentatives restantes
        return: l'objet Response si succès, sinon None
        """

        try:
            # On tente la requête
            response = requests.get(url, headers=self.headers, timeout=timeout)

            # Si le code HTTP indique un succès (200, 201, etc.)
            response.raise_for_status()  # lève une erreur pour les codes 4xx/5xx

            return response

        except (requests.exceptions.RequestException) as e:
            # En cas d'erreur réseau, timeout, mauvais code HTTP,...
            print(f"Erreur lors de la requête vers {url} : {e}")

            if retries > 0:
                print(f"Nouvelle tentative... (retries restants : {retries})")
                # Appel récursif avec un retry en moins
                return self.get(url, timeout=timeout, retries=retries - 1)
            else:
                print("Plus de retries disponibles, abandon.")
                return None



## Exercice 2

- Faire une fonction permettant de supprimer tous les espaces supperflus d'une string
- Faire une fonction qui prend une string html et renvois une string intelligible (enlever les caractères spéciaux,
- Récupérer le domaine en fonction d'un url

In [7]:
# pour retirer les esapces présent dans un string
def clean_spaces(text: str) -> str:
    """
    Supprime les espaces superflus :
    - espaces en début/fin
    - plusieurs espaces consécutifs -> un seul
    """
    # fonction .split() coupe sur n'importe quel espace et enlève les doublons
    parts = text.split()
    # on rejoint avec un seul espace
    return " ".join(parts)


In [8]:
import re
import html

def html_to_text_simple(html_string: str) -> str:
    """
    Transforme une string HTML en texte lisible
    """
    # Enlever les balises <...>
    #    <[^>]+> veut dire : un '<', puis n'importe quels caractères sauf '>', puis un '>' 
    # on remplace par un espace pour éviter de coller des mots ensemble
    text = re.sub(r"<[^>]+>", " ", html_string)

    # Décoder les entités HTML (&amp; -> &, &eacute; -> é, etc.)
    # on utilise la bibliothèque standard html
    text = html.unescape(text)

    # Nettoyer les espaces superflus avec la fonction définie plus haut
    text = clean_spaces(text)

    return text


In [9]:
from urllib.parse import urlparse

def get_domain(url: str) -> str:
    """
    Récupère le domaine à partir d'une URL complète.
    """
    # on utilise la fonction urlparse pour analyser l'URL
    parsed = urlparse(url)
    domain = parsed.netloc  # ex: 'www.google.com'

    # Optionnel : enlever 'www.' au début
    if domain.startswith("www."):
        domain = domain[4:]
    
    return domain


# Exploitation du HTML  

Ici, il faut récupérer le code HTML d'un site web à partir d'une requête. Lorsque vous avez récupéré le texte d'un site il faut le parser. Pour cela, on utilise BeautifulSoup qui permet de transformer la structure HTML en objet Python. Cela permet de récupérer efficacement les données qui nous intéresse.

Pour les webmasters, le blocage le plus souvent mis en place et un blocage sur le User-Agent. Le User-Agent est un paramètre intégré dans la requête HTTP réalisé par le Navigateur pour envoyer au front des informations basiques :

- la version du Navigateur,
- la version de l'OS
- Le type de gestionnaire graphique (Gecko)
- le type de device utilisé

Exemple de User Agent :  

`Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0`

Commençons à utiliser `BeautifulSoup`, il est normalement déjà installé, le cas échéant executez les lignes suivantes : 

In [10]:
import requests
from bs4 import BeautifulSoup

Pour transformer une requête (requests) en objet BeautifulSoup :

In [11]:
response = requests.get(url)
soup = BeautifulSoup(response.text)

Pour trouver tous les liens d'une page, on récupère la balise `a` qui permet de gérer les liens en HTML :

In [12]:
soup.find_all("a")[0:10]

[<a href="/#content">Aller au contenu</a>,
 <a href="/#menu">Aller au menu</a>,
 <a href="/plan-du-site/">Plan du site</a>,
 <a href="/actualites/journees-portes-ouvertes-2025-2026" target="_blank" title="Ouvre une nouvelle fenêtre">Journée portes ouvertes le 6 décembre de 13h à 18h. Inscrivez-vous dès maintenant !</a>,
 <a href="/"><img alt="ESIEE PARIS" class="a42-ac-replace-img" src="/typo3conf/ext/esiee_sitepackage/Resources/Public/imgs/svg/logo-esiee.svg"/></a>,
 <a href="/brochures-1">Brochures</a>,
 <a href="/informations/etudiantes-et-etudiants">Espace élèves</a>,
 <a href="/" hreflang="fr-FR" title="Français">
 <span>Fr</span>
 </a>,
 <a href="/en/" hreflang="en-US" title="English">
 <span>En</span>
 </a>,
 <a href="/candidater-1">Candidater</a>]

On peut aussi préciser la classe HTML qu'on veut récupérer :

```python
soup.find_all(class_="<CLASS_NAME>")[0:10]
```

Ici par exemple: 

In [13]:
soup.find_all(class_="toggler")[0:5]

[<button aria-controls="searchbox-header-form" aria-expanded="false" class="toggler">
 <i class="fa-solid fa-magnifying-glass"></i>
 <i class="fa-solid fa-xmark"></i>
 <span class="sr-only">
 <span class="display">Afficher</span><span class="hide">Masquer</span> la recherche
 		</span>
 </button>,
 <button aria-controls="submenu-40" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>L'école</button>,
 <button aria-controls="submenu-563" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>Gouvernance et conseils</button>,
 <button aria-controls="submenu-65" aria-expanded="false" class="toggler"><span class="sr-only"><span class="display">Afficher</span><span class="hide">Masquer</span> le sous menu : </span>Départements d'enseignements et de recherche</button>,
 <button aria-controls="su

Pour récupérer le text sans les balises HTML :

In [14]:
soup.text[0:1000]

"\n\n\n\nESIEE Paris, l'école de l'innovation technologique | ESIEE Paris\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAller au contenu\nAller au menu\nPlan du site\n\n\n\n\n\n\n\nJournée portes ouvertes le 6 décembre de 13h à 18h. Inscrivez-vous dès maintenant !\n\n\n\n\n\nMasquer l'alerte\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBrochuresEspace élèves\n\n\n\nFr\n\n\n\n\nEn\n\n\n\n\n\n\n\n\n\nAfficherMasquer la recherche\r\n\t\t\n\n\n\nSaisissez votre recherche\xa0:\n\nLancer la recherche\n\n\n\nCandidater\n\nAfficherMasquer le menu\n\n\n\n\n\nRetour au menu principalAfficherMasquer le sous menu\xa0: L'écolePourquoi choisir ESIEE Paris ?AfficherMasquer le sous menu\xa0: Gouvernance et conseilsGouvernance et conseilsConseil scientifiqueAfficherMasquer le sous menu\xa0: Départements d'enseignements et de rechercheInformatique et télécommunicationsIngénierie des systèmes cyberphysiquesIngénierie industrielleSanté, énergie et environnement durableManagement, sciences humaines et languesCorps pr

## Exercice
### Exercice 3

Améliorer la classe développé précédemment.

- Ajouter une méthode pour récupérer l'objet soup d'un url
- Récupérer une liste de User Agent et effectuer une rotation aléatoire sur celui à utiliser
- Utiliser cette classe pour parser une page HTML et récupérer : le titre, tous les H1 (si ils existent), les liens vers les images, les liens sortants vers d'autres sites, et le texte principal.

In [12]:
import random
from urllib.parse import urljoin

class HttpClient:
    def __init__(self, user_agents=None):
        """
        user_agents : liste de chaînes User-Agent.
        Si None, on utilise une petite liste par défaut.
        """
        if user_agents is None:
            self.user_agents = [
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
                "Mozilla/5.0 (X11; Linux x86_64)",
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
                "MonSuperScraper/1.0"
            ]
        else:
            self.user_agents = user_agents

    def _get_random_headers(self):
        """
        Choisit un User-Agent au hasard et construit les headers.
        """
        ua = random.choice(self.user_agents)
        return {"User-Agent": ua}

    def get(self, url: str, timeout: float = 5.0, retries: int = 3):
        """
        Effectue une requête HTTP GET avec :
        - rotation aléatoire de User-Agent
        - timeout
        - retries récursifs
        """
        try:
            headers = self._get_random_headers()
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()  # lève une exception si code HTTP 4xx/5xx
            return response

        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la requête vers {url} : {e}")

            if retries > 0:
                print(f"Nouvelle tentative... (retries restants : {retries})")
                return self.get(url, timeout=timeout, retries=retries - 1)
            else:
                print("Plus de retries disponibles, abandon.")
                return None
    
    def get_soup(self, url: str, timeout: float = 5.0, retries: int = 3):
        """
        Récupère un objet BeautifulSoup à partir d'une URL.
        """
        response = self.get(url, timeout=timeout, retries=retries)
        if response is None:
            return None

        # Crée l'objet soup à partir du HTML
        soup = BeautifulSoup(response.text, "html.parser")
        return soup
    
    def parse_page(self, url: str, timeout: float = 5.0, retries: int = 3):
        """
        Parse une page HTML et renvoie :
        - title : titre de la page
        - h1_list : liste de tous les H1
        - image_links : liste des URLs d'images
        - external_links : liste des liens sortants vers d'autres domaines
        - main_text : texte principal (contenu des <p>)
        """
        soup = self.get_soup(url, timeout=timeout, retries=retries)
        if soup is None:
            return None

        # ----- Titre -----
        title_tag = soup.find("title")
        title = title_tag.get_text(strip=True) if title_tag else ""

        # ----- H1 -----
        h1_list = [h.get_text(strip=True) for h in soup.find_all("h1")]

        # ----- Images -----
        image_links = []
        for img in soup.find_all("img"):
            src = img.get("src")
            if src:
                full_url = urljoin(url, src)
                image_links.append(full_url)

        # ----- Liens sortants -----
        parsed_base = urlparse(url)
        base_domain = parsed_base.netloc.replace("www.", "")

        external_links_set = set()
        for a in soup.find_all("a", href=True):
            href = a["href"]
            full_url = urljoin(url, href)
            parsed_link = urlparse(full_url)

            if parsed_link.scheme not in ("http", "https"):
                continue

            link_domain = parsed_link.netloc.replace("www.", "")
            if link_domain and link_domain != base_domain:
                external_links_set.add(full_url)

        external_links = list(external_links_set)

        # ----- Texte principal -----
        paragraphs = [p.get_text(" ", strip=True) for p in soup.find_all("p")]
        main_text = " ".join(paragraphs)
        main_text = clean_spaces(main_text)

        # On renvoie tout dans un dictionnaire
        return {
            "title": title,
            "h1_list": h1_list,
            "image_links": image_links,
            "external_links": external_links,
            "main_text": main_text
        }



# Exploitation des appels d'API



Losque le front du site récupère des données sur une API gérée par le back, un appel d'API est réalisé. Cet appel est recensé dans les appels réseaux. Il est alors possible de re-jouer cet appel pour récupérer à nouveau les données. Il est très facile de récupérer ces appels dans l'onglet Network de la console développeur de Chrome ou FireFox. La console vous permet de copier le code CURL de la requête et vous pouvez ensuite la transformer en code Python depuis le site https://curl.trillworks.com/.

Souvent les APIs sont bloquées avec certains paramètres. L'API vérifie que dans les headers de la requête HTTP ces paramètres sont présents :
* un token généré à la volée avec des protocoles OAuth2 (ou moins développés).
* un referer provenant du site web (la source de la requête), très facile à falsifier.



## Exercice 
### Exercice 4

- Utiliser les informations développées plus haut pour récupérer les premiers résultats d'une recherche d'une requête
sur Google. 

Tips : 

- Ouvrir les outils de développements de Chrome ou Firefox
- Onglet Network
- Fouiller dans les requêtes pour voir à quoi ressemble un appel API Google
- Utilisez beautiful soup pour convertir le contenu de la request en objet et accéder aux balises

In [ ]:
def get_soup(self, url: str, timeout: float = 5.0, retries: int = 3, parser: str = "html.parser"):
        """
        Récupère un objet BeautifulSoup à partir d'une URL.
        parser : "html.parser" (par défaut) ou "xml" pour les flux RSS.
        """
        response = self.get(url, timeout=timeout, retries=retries)
        if response is None:
            return None

        soup = BeautifulSoup(response.text, parser)
        return soup

In [15]:
from urllib.parse import quote_plus

def build_google_search_url(query: str, lang: str = "fr") -> str:
    """
    Construit une URL de recherche Google à partir d'une requête.
    """
    q = quote_plus(query)  # remplace espaces par +, etc.
    return f"https://www.google.com/search?q={q}&hl={lang}"


In [16]:
def google_search(client: HttpClient, query: str, max_results: int = 5):
    """
    Utilise la classe HttpClient pour récupérer les premiers résultats Google d'une requête.
    Retourne une liste de dictionnaires
    """
    # Construire l'URL de recherche avec la fonction définie plus haut
    search_url = build_google_search_url(query)

    #Récupérer le soup avec notre client
    soup = client.get_soup(search_url)
    if soup is None:
        print("Impossible de récupérer la page de résultats Google")
        return []

    results = []

    # Trouver les blocs de résultats : div.g 
    for result_block in soup.select("div.g"):
        # Titre
        title_tag = result_block.find("h3")
        if not title_tag:
            continue  # si pas de titre, on saute ce bloc

        title = title_tag.get_text(strip=True)

        # Lien
        link_tag = result_block.find("a", href=True)
        if not link_tag:
            continue

        url = link_tag["href"]

        # Snippet 
        snippet_tag = result_block.find("div", class_="VwiC3b")
        if snippet_tag is None:
            snippet_tag = result_block.find("span", class_="aCOpRe")

        snippet = ""
        if snippet_tag is not None:
            snippet = snippet_tag.get_text(" ", strip=True)

        results.append({
            "title": title,
            "url": url,
            "snippet": snippet
        })

        if len(results) >= max_results:
            break

    return results


# Exercice Final  

Exercice Final
Utilisez tout ce que vous avez appris pour récupérer des articles de News avec une catégorie. Il est souvent intéressant de partir des flux RSS pour commencer :

Les données doivent comprendre :
- Le texte important propre
- L'url
- Le domaine
- la catégorie
- Le titre de l'article
- Le titre de la page
- (Facultatif) : les images

Tips : 

- Taper le nom de votre média favoris + RSS (par exemple : https://www.lemonde.fr/rss/)
- Aller dans le DOM de la page 
- Trouver les catégories et les liens vers les articles

In [18]:
class HttpClient:
    def __init__(self, user_agents=None):
        if user_agents is None:
            self.user_agents = [
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
                "Mozilla/5.0 (X11; Linux x86_64)",
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
                "MonSuperScraper/1.0"
            ]
        else:
            self.user_agents = user_agents

    def _get_random_headers(self):
        ua = random.choice(self.user_agents)
        return {"User-Agent": ua}

    def get(self, url: str, timeout: float = 5.0, retries: int = 3):
        try:
            headers = self._get_random_headers()
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la requête vers {url} : {e}")
            if retries > 0:
                print(f"Nouvelle tentative... (retries restants : {retries})")
                return self.get(url, timeout=timeout, retries=retries - 1)
            else:
                print("Abandon.")
                return None

    def get_soup(self, url: str, timeout: float = 5.0, retries: int = 3, parser: str = "html.parser"):
        response = self.get(url, timeout=timeout, retries=retries)
        if response is None:
            return None
        return BeautifulSoup(response.text, parser)


In [19]:
def fetch_rss_items(client: HttpClient, rss_url: str, max_items: int = 5):
    """
    Récupère les items d'un flux RSS (titre, lien, catégorie éventuelle).
    Utilise HttpClient.get_soup avec parser="xml".
    """
    soup = client.get_soup(rss_url, parser="xml")
    if soup is None:
        print("Impossible de récupérer le flux RSS.")
        return []

    items = []
    for item in soup.find_all("item")[:max_items]:
        title_tag = item.find("title")
        link_tag = item.find("link")
        category_tag = item.find("category")

        title = title_tag.get_text(strip=True) if title_tag else ""
        link = link_tag.get_text(strip=True) if link_tag else ""
        category = category_tag.get_text(strip=True) if category_tag else ""

        items.append({
            "rss_title": title,
            "url": link,
            "category": category
        })

    return items


In [20]:
def parse_article_page(client: HttpClient, article_info: dict):
    """
    Prend un dict contenant au moins:
    - 'url'
    - 'category' (depuis le RSS, peut être vide)
    - 'rss_title'

    Retourne un dict avec :
    - text : texte important propre
    - url
    - domain
    - category
    - article_title
    - page_title
    - images
    """
    url = article_info["url"]
    category = article_info.get("category", "")
    rss_title = article_info.get("rss_title", "")

    # Si pas de catégorie dans le RSS, on met une valeur par défaut
    if not category:
        category = "Ouest-France"

    soup = client.get_soup(url)
    if soup is None:
        return None

    # Titre de la page (<title>)
    page_title_tag = soup.find("title")
    page_title = page_title_tag.get_text(strip=True) if page_title_tag else ""

    # Titre de l'article (souvent <h1>)
    h1_tag = soup.find("h1")
    article_title = h1_tag.get_text(strip=True) if h1_tag else rss_title

    # Texte principal : on essaie d'abord <article>, sinon tous les <p>
    article_tag = soup.find("article")
    if article_tag:
        paragraphs = [p.get_text(" ", strip=True) for p in article_tag.find_all("p")]
    else:
        paragraphs = [p.get_text(" ", strip=True) for p in soup.find_all("p")]

    main_text = clean_spaces(" ".join(paragraphs))

    # Images : src des <img>
    image_links = []
    img_tags = article_tag.find_all("img") if article_tag else soup.find_all("img")
    for img in img_tags:
        src = img.get("src")
        if src:
            full_url = urljoin(url, src)
            image_links.append(full_url)

    # Domaine (ouest-france.fr)
    parsed = urlparse(url)
    domain = parsed.netloc
    if domain.startswith("www."):
        domain = domain[4:]

    return {
        "text": main_text,
        "url": url,
        "domain": domain,
        "category": category,
        "article_title": article_title,
        "page_title": page_title,
        "images": image_links
    }


In [21]:
def fetch_news_from_rss(client: HttpClient, rss_url: str, max_items: int = 5):
    """
    1) Récupère quelques items depuis le flux RSS
    2) Pour chaque item, va sur la page de l'article et la parse
    3) Retourne une liste de dicts complets
    """
    rss_items = fetch_rss_items(client, rss_url, max_items=max_items)

    articles = []
    for item in rss_items:
        parsed = parse_article_page(client, item)
        if parsed is not None:
            articles.append(parsed)
    return articles


In [22]:
client = HttpClient()

# 👉 Flux RSS Ouest-France (France entière)
rss_url = "https://www.ouest-france.fr/rss.xml?insee=FRA"

articles = fetch_news_from_rss(client, rss_url, max_items=3)

for i, art in enumerate(articles, start=1):
    print("="*80)
    print(f"ARTICLE {i}")
    print("Titre de la page   :", art["page_title"])
    print("Titre de l'article :", art["article_title"])
    print("URL                :", art["url"])
    print("Domaine            :", art["domain"])
    print("Catégorie          :", art["category"])
    print("Nb images trouvées :", len(art["images"]))
    print("\nTexte (début) :")
    print(art["text"][:500], "...\n")


ARTICLE 1
Titre de la page   : Après Shein, la plateforme eBay visée par une enquête pour vente de produits illégaux
Titre de l'article : Après Shein, la plateforme eBay visée par une enquête pour vente de produits illégaux
URL                : https://www.ouest-france.fr/economie/commerce/e-commerce/apres-shein-la-plateforme-ebay-visee-par-une-enquete-pour-vente-de-produits-illegaux-e6c4e986-c9ff-11f0-a0e6-83b9718ad3c0?utm_source=fluxpublicfrance&utm_medium=fluxrss&utm_campaign=banquedecontenu
Domaine            : ouest-france.fr
Catégorie          : Ouest-France
Nb images trouvées : 2

Texte (début) :
Mardi 25 novembre, le parquet de Paris a indiqué que la plateforme américaine eBay était visée par une enquête pour vente de produits illégaux. Quatre autres plateformes, Shein, Temu, Wish et AliExpress sont déjà poursuivies en France. Ouest-France Newsletter Mon Budget Chaque semaine, infos pratiques et conseils utiles pour vos dépenses du quotidien Merci de saisir votre adresse e-mail